In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Flatten, TimeDistributed, Dropout ,Bidirectional, Input, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from glob import glob
import cv2


# Data Loading & Preparation


In [3]:
! pip install opendatasets

In [4]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/yasharjebraeily/drowsy-detection-dataset")

Skipping, found downloaded files in "./drowsy-detection-dataset" (use force=True to force download)


In [5]:
# Set parameters
IMG_SIZE = (224, 224)
SEQUENCE_LENGTH = 10  # Number of frames in sequence
BATCH_SIZE = 16
EPOCHS = 20

In [6]:
DATASET_PATH = "/content/drowsy-detection-dataset/Drowsy_datset"
TRAIN_PATH = os.path.join(DATASET_PATH, "train")
TEST_PATH = os.path.join(DATASET_PATH, "test")

In [7]:
# Verify and access class directories
sub_dirs = []
if os.path.isdir(DATASET_PATH):
    sub_dirs = [os.path.join(DATASET_PATH, sub_dir) for sub_dir in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, sub_dir))]
    print("Subdirectories inside dataset:", sub_dirs)
    for sub_dir in sub_dirs:
        files = os.listdir(sub_dir)
        print(f"Files in '{sub_dir}':", files[:4])

Subdirectories inside dataset: ['/content/drowsy-detection-dataset/Drowsy_datset/train', '/content/drowsy-detection-dataset/Drowsy_datset/test']
Files in '/content/drowsy-detection-dataset/Drowsy_datset/train': ['NATURAL', 'DROWSY']
Files in '/content/drowsy-detection-dataset/Drowsy_datset/test': ['NATURAL', 'DROWSY']


In [15]:
# Image Data Generator for efficient loading
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=IMG_SIZE,
    batch_size=1,  # Generate one image at a time for sequences
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=IMG_SIZE,
    batch_size=1,  # Generate one image at a time for sequences
    class_mode='categorical',
    subset='validation'
)

# Custom sequence generator
def sequence_generator(image_generator, batch_size, timesteps):
    while True:
        batch_images = []
        batch_labels = []
        for _ in range(batch_size):
            sequence_images = []
            for _ in range(timesteps):
                img, label = next(image_generator)
                sequence_images.append(img[0])  # Extract image from batch
            batch_images.append(np.stack(sequence_images, axis=0))
            batch_labels.append(label[0])  # Extract label from batch
        yield np.array(batch_images), np.array(batch_labels)

train_sequence_generator = sequence_generator(train_generator, BATCH_SIZE, SEQUENCE_LENGTH)
val_sequence_generator = sequence_generator(val_generator, BATCH_SIZE, SEQUENCE_LENGTH)


Found 4688 images belonging to 2 classes.
Found 1171 images belonging to 2 classes.


# CNN(VGG16 FineTuned)

FineTuned VGG16

CNN-LSTM Model

In [16]:
# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Unfreeze last few layers for fine-tuning
for layer in base_model.layers[-6:]:
    layer.trainable = True

# Feature extractor model
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

# Build CNN-LSTM Model
model = Sequential([
    TimeDistributed(feature_extractor, input_shape=(SEQUENCE_LENGTH, 224, 224, 3)),
    TimeDistributed(Flatten()),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')  # Multi-class classification
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(
    train_sequence_generator,
    steps_per_epoch=len(train_generator) // BATCH_SIZE,
    validation_data=val_sequence_generator,
    validation_steps=len(val_generator) // BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 677s 2s/step - accuracy: 0.5106 - loss: 0.7096 - val_accuracy: 0.4546 - val_loss: 0.7066
Epoch 2/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 683s 2s/step - accuracy: 0.5155 - loss: 0.6959 - val_accuracy: 0.4863 - val_loss: 0.6934
Epoch 3/20
124/293 ━━━━━━━━━━━━━━━━━━━━ 5:49 2s/step - accuracy: 0.5203 - loss: 0.6927